<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/TF_Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создаем простой граф


In [0]:
import tensorflow as tf

x = tf.Variable (3, name="x")
y = tf.Variable (4, name="y")
f = x*x*y + y + 2


**Добавляем сессию**

In [0]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


**Упращенная сессия (с автозакрытием)**

In [0]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()
  print(result)

42


**Глобальная инициализация**

In [0]:
import tensorflow as tf

x = tf.Variable(3, "x")
y = tf.Variable(2, "y")
z = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
  init.run()
  result = f.eval()
  print(result)

42


**Новый отдельный экземпляр графа **

In [0]:
graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)
  y2 = tf.Variable(6)
  Z2 = x2*y2
  
  init = tf.global_variables_initializer()
  sess = tf.InteractiveSession()
  init.run()
  result2 = Z2.eval()
  print(result2)
  sess.close()
  

12


**Оценка в одном прогоне**

In [0]:
import tensorflow as tf
w = tf.constant(3)
x = w + 2
y = x + 5 
z = x * 3

init = tf.global_variables_initializer()
with tf.Session() as sess:
  init.run()
  y_val, z_val = sess.run([y,z])
  print(y_val)
  print(z_val)
 
  sess.close()


10
15


**Линейная регресия упрощенная**

In [0]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()   
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape

#print(housing.feature_names)
#print(housing.DESCR)
#print(housing.target.shape)

# np.ones создает новый array заполненный 1ми, np.c_ создает новый масив обьединяя 2
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant (housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant (housing.target.reshape(-1,1), dtype=tf.float32, name="y")

XT = tf.transpose(X)
theta = tf.matmul (tf.matmul (tf.matrix_inverse(tf.matmul(XT,X)), XT) , y)

with tf.Session() as sess:
  theta_value = theta.eval ()
  print(theta_value)

NameError: ignored

**Вычисление градиента в ручную и автоматически**

In [0]:




import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

m, n = housing.data.shape

n_epochs = 100000
learning_rate = 0.0001

# нормализация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

# np.c_ сращивание матриц, np.ones создание матрицы заполненой нулями
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# tf.random_uniform генерировать случайный тензер с заданой формой и диапазоном
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") 
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y

# tf.reduce_mean вычисляет среднее значение
mse = tf.reduce_mean(tf.square(error), name="mse")

#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients (mse, [theta])[0]

# создает узел который будет присваивать переменной новое значение 
training_op = tf.assign(theta, theta - learning_rate * gradients) 
init = tf.global_variables_initializer()
 

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
           k = y_pred.eval()
           print("Epoch", epoch, "MSE =", mse.eval(), "y_pred", y_pred.shape, "theta", theta.shape, "error", error.shape, "X", X.shape)
        sess.run(training_op)
    
    best_theta = theta.eval()

 **Вывод матрицы/тензора **

In [0]:
import tensorflow as tf
g = tf.random_uniform([10 + 1, 1], 0.0, 10.0, seed=42)
init_op = tf.initialize_all_variables()
h = tf.transpose(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42))
k = tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42)
with tf.Session() as sess:
    sess.run(init_op) 
    print (sess.run(h))#распечатать тензор
    print (sess.run(k))#распечатать тензор
    print(g.get_shape) #получить размер массива статич

       




[[ 0.9045429   0.35481548  0.5906365   0.51156354 -0.04808879  0.26202965
  -0.62795925 -0.7713845  -0.32755637]]
[[ 0.9045429 ]
 [ 0.35481548]
 [ 0.5906365 ]
 [ 0.51156354]
 [-0.04808879]
 [ 0.26202965]
 [-0.62795925]
 [-0.7713845 ]
 [-0.32755637]]
<bound method Tensor.get_shape of <tf.Tensor 'random_uniform_22:0' shape=(11, 1) dtype=float32>>


**Использование оптимизатора**

In [1]:
import tensorflow as tf
import numpy as np


from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler


housing = fetch_california_housing()
m, n = housing.data.shape

learning_rate = 0.0001
n_epochs = 10000

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

def fetch_batch (batch_index, batch_size, epoch):
    np.random.seed (epoch * batch_size + batch_index) 
    indices = np.random.randint (m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias [indices]
    y_batch = housing.target.reshape(-1, 1) [indices]
    return X_batch, y_batch




In [0]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="prediction")
erorr = y_pred - y
mse = tf.reduce_mean(tf.square(erorr), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), erorr, name="gradients")
#training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.AdamOptimizer (learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

batch_size = 1000
n_batches = int(np.ceil(m / batch_size))

init = tf.global_variables_initializer()
with tf.Session() as sess:
     sess.run(init)
     for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch (batch_index, batch_size, epoch)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 100 == 0 :
            print("MSE=%s" % sess.run(mse, feed_dict={X:X_batch, y:y_batch}))
            print("epoch", epoch)
            #_, mse_value = sess.run([training_op, mse], feed_dict={X:X_batch, y:y_batch})
            #print("mse_value=%s" % mse_value)
            


In [10]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

learning_rate = 0.001
epoch = 1000
batch_size = 1000


housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
california_housing_scaler = scaler.fit_transform(housing.data)

california_housing_scaler_with_bias = np.c_[np.ones((m, 1)), california_housing_scaler]

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta)

with tf.name_scope("lose") as scope:
     erorr = y_pred - y
     mse = tf.reduce_mean(tf.square(erorr), name="MSE")

optimizer = tf.train.AdamOptimizer (learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
n_batches = int(np.ceil(m / batch_size))

def fitch_batch (batch_size, batch_n, epoch_n):
    np.random.seed(batch_size * epoch_n + batch_n)
    indexed = np.random.randint(m, size=batch_size)
    X_batch = california_housing_scaler_with_bias [indexed]
    y_batch = housing.target.reshape(-1, 1) [indexed]
    return X_batch, y_batch
saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(init)
  #saver.restore (sess, "/tmp/my_model_fin.ckpt")
  for epoch_n in range(epoch):
    for batch_n in range(n_batches):
      X_batch, y_batch = fitch_batch (batch_size, batch_n, epoch_n)
      sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    if epoch_n % 100 == 0:
      mse_p = sess.run(mse, feed_dict={X:X_batch, y:y_batch})
      print(mse_p)
      save_path = saver.save(sess, "/tmp/my_model.ckpt")
    
  save_path = saver.save(sess, "/tmp/my_model_fin.ckpt") 


6.583357
0.5733535
0.5721193
0.51688707
0.5379817
0.4978484
0.5438165
0.48000878
0.5129886
0.5052237


In [13]:
#вывод графа
saver = tf.train.Saver()
with tf.Session() as sess:
  saver.restore (sess, "/tmp/my_model_fin.ckpt")
  print(mse_p)
  
print(mse.op.name)
#show_graph(tf.get_default_graph())

INFO:tensorflow:Restoring parameters from /tmp/my_model_fin.ckpt
0.5052237
lose/MSE


In [0]:
#функция для вывода графа
#from tensorflow_graph_in_jupyter import show_graph
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))